## HMM

### Code

#### 向前/向后算法

In [ ]:
import numpy as np

In [ ]:
class Hidden_Markov_Model:
    def forward(self, A, B, PI, Q, V, O):
        Q_n = len(Q)
        O_n = len(O)
        alphas = np.zeros((O_n, Q_n))
        for t in range(O_n):
            index = V.index(O[t])
            if t == 0:
                alphas[t] = PI * B[:,index]
            else:
                alphas[t] = np.dot(alphas[t-1], A) * B[:,index]
        P = np.sum(alphas[O_n-1])
        print("P(O|lambda) =", P)
        return alphas
        
    def backward(self, A, B, PI, Q, V, O):
        Q_n = len(Q)
        O_n = len(O)
        betas = np.ones((O_n, Q_n))
        for t in range(O_n-2,-1,-1):
            index = V.index(O[t+1])
            betas[t] = np.dot(A * B[:,index], betas[t+1])
        P = np.dot(PI * B[:,V.index(O[0])], betas[0])
        print("P(O|lambda) =", P)
        return betas

#### 测试

对于10.1习题例子（主要根据前向和后向算法计算结果）：

In [ ]:
A = np.array([[0.5,0.2,0.3],[0.3,0.5,0.2],[0.2,0.3,0.5]])
B = np.array([[0.5,0.5],[0.4,0.6],[0.7,0.3]])
PI = np.array([0.2,0.4,0.4])
V = ["red","white"]
Q = [1,2,3]
O = ["red","white","red","white"]
        
HMM = Hidden_Markov_Model()
HMM = Hidden_Markov_Model()
print("P(O|lambda) =", HMM.forward(A,B,PI,Q,V,O))
print("P(O|lambda) =", HMM.backward(A,B,PI,Q,V,O))

对于10.2习题例子（根据公式10.24公式，加上前向和后向概率矩阵，从而计算结果）：

In [ ]:
A = np.array([[0.5,0.1,0.4],[0.3,0.5,0.2],[0.2,0.2,0.6]])
B = np.array([[0.5,0.5],[0.4,0.6],[0.7,0.3]])
PI = np.array([0.2,0.3,0.5])
V = ["red","white"]
Q = [1,2,3]
O = ["red","white","red","red","white","red","white","white"]

HMM = Hidden_Markov_Model()
alpha = HMM.forward(A,B,PI,Q,V,O)
beta = HMM.backward(A,B,PI,Q,V,O)
p = alpha[3,2]*beta[3,2]/sum(alpha[3] * beta[3])
print("P(i4=q3|O,lambda) = ", p)

#### 维特比算法

In [ ]:
class Hidden_Markov_Model:
    def vertibi(self, A, B, PI, Q, V, O):
        Q_n = len(Q)
        O_n = len(O)
        deltas = np.zeros((O_n, Q_n))
        psis = np.zeros((O_n, Q_n))
        I = np.zeros(O_n)
        for t in range(O_n):
            index = V.index(O[t])
            for i in range(Q_n):
                if t == 0:
                    deltas[t,i] = PI[i] * B[i,index]
                    psis[t,i] = 0
                else:
                    deltas[t,i] = np.max(A[:,i] * deltas[t-1,:]) * B[i,index]
                    psis[t,i] = np.argmax(A[:,i] * deltas[t-1,:]) + 1
        I[O_n-1] = np.argmax(deltas[O_n-1,:]) + 1
        for t in range(O_n-2,-1,-1):
            I[t] = psis[t+1,int(I[t+1])-1]
        print("I =", I)
        return

#### 测试

以习题10-3为例：

In [ ]:
A = np.array([[0.5,0.2,0.3],[0.3,0.5,0.2],[0.2,0.3,0.5]])
B = np.array([[0.5,0.5],[0.4,0.6],[0.7,0.3]])
PI = np.array([0.2,0.4,0.4])
V = ["red","white"]
Q = [1,2,3]
O = ["red","white","red","white"]

HMM = Hidden_Markov_Model()
HMM.vertibi(A,B,PI,Q,V,O)
#>>I = [3. 2. 2. 2.]